In [1]:
import pandas as pd
import numpy as np
import pickle
import gc

import warnings
warnings.filterwarnings('ignore')

# load data

In [47]:
df_properties_2016 = pd.read_csv('../data/properties_2016.csv')
df_properties_2017 = pd.read_csv('../data/properties_2017.csv')

In [48]:
df_transactions_2016 = pd.read_csv('../data/train_2016_v2.csv')
df_transactions_2017 = pd.read_csv('../data/train_2017.csv')

In [49]:
sample_submission = pd.read_csv('../data/sample_submission.csv', low_memory = False)

In [50]:
df_train_2016 = pd.merge(df_transactions_2016, df_properties_2016, how = 'left', on = 'parcelid')
df_train_2017 = pd.merge(df_transactions_2017, df_properties_2017, how = 'left', on = 'parcelid')

# assign 2017 tax data to NULL due to info leak
# df_train_2017[['structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxvaluedollarcnt', 'taxamount']] = np.nan

# merge the two set 
df_train = pd.concat([df_train_2016, df_train_2017], axis = 0)

In [45]:
df_pred_2016 = pd.merge(sample_submission[['ParcelId']], df_properties_2016.rename(columns = {'parcelid': 'ParcelId'}), 
                how = 'left', on = 'ParcelId')
df_pred_2017 = pd.merge(sample_submission[['ParcelId']], df_properties_2017.rename(columns = {'parcelid': 'ParcelId'}), 
                how = 'left', on = 'ParcelId')

In [46]:
# get some memory back
del df_properties_2016, df_properties_2017, df_train_2016, df_train_2017
gc.collect()

338

# Pre-process data

## feature engineering

In [35]:
d_features.head()

,month,avg_logerror,avg_abs_logerror,std_dev_logerror,std_dev_abs_logerror
0,1,0.019012,0.073773,0.172725,0.157330
1,2,0.018463,0.074905,0.181878,0.166761
2,3,0.012199,0.070451,0.163923,0.148513
3,4,0.008849,0.069463,0.162658,0.147345
4,5,0.009161,0.067088,0.157046,0.142290


### features apply to all data

In [51]:
def add_date_features(df):
    #df["transaction_year"] = df["transactiondate"].dt.year
    #df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    
    df['transaction_month'] = pd.to_datetime(t.transactiondate).dt.month
    df["transaction_quarter"] = pd.to_datetime(t.transactiondate).dt.quarter
    
    # df.drop(["transactiondate"], inplace=True, axis=1)

    #error in calculation of the finished living area of home
    df['f_num_LivingAreaError'] = df['calculatedfinishedsquarefeet']/df['finishedsquarefeet12']

    #proportion of living area
    df['f_num_LivingAreaProp'] = df['calculatedfinishedsquarefeet']/df['lotsizesquarefeet']
    df['f_num_LivingAreaProp2'] = df['finishedsquarefeet12']/df['finishedsquarefeet15']

    #Amout of extra space
    df['f_num_ExtraSpace'] = df['lotsizesquarefeet'] - df['calculatedfinishedsquarefeet'] 
    df['f_num_ExtraSpace-2'] = df['finishedsquarefeet15'] - df['finishedsquarefeet12'] 

    #Total number of rooms
    df['f_num_TotalRooms'] = df['bathroomcnt']*df['bedroomcnt']

    #Average room size
    df['f_num_AvRoomSize'] = df['calculatedfinishedsquarefeet']/df['roomcnt'] 

    # Number of Extra rooms
    df['f_num_ExtraRooms'] = df['roomcnt'] - df['f_num_TotalRooms'] 

    #Ratio of the built structure value to land area
    df['f_num_ValueProp'] = df['structuretaxvaluedollarcnt']/df['landtaxvaluedollarcnt']

    #Does property have a garage, pool or hot tub and AC?
    df['f_num_GarPoolAC'] = ((df['garagecarcnt']>0) & (df['pooltypeid10']>0) & (df['airconditioningtypeid']!=5))*1 

    df["f_num_location"] = df["latitude"] + df["longitude"]
    df["f_num_locatiof_num_2"] = df["latitude"]*df["longitude"]
    df["f_num_locatiof_num_2round"] = df["f_num_locatiof_num_2"].round(-4)

    df["f_num_latitude-round"] = df["latitude"].round(-4)
    df["f_num_longitude-round"] = df["longitude"].round(-4)
    
    #polnomials of the variable
    df["f_num_structuretaxvaluedollarcnt-2"] = df["structuretaxvaluedollarcnt"] ** 2
    df["f_num_structuretaxvaluedollarcnt-3"] = df["structuretaxvaluedollarcnt"] ** 3

    #Ratio of tax of property over parcel
    df['ValueRatio'] = df['taxvaluedollarcnt']/df['taxamount']

    #TotalTaxScore
    df['TaxScore'] = df['taxvaluedollarcnt']*df['taxamount']

    #polnomials of tax delinquency year
    df["taxdelinquencyyear-2"] = df["taxdelinquencyyear"] ** 2
    df["taxdelinquencyyear-3"] = df["taxdelinquencyyear"] ** 3

    return df

In [53]:
df_train_2016 = add_date_features(df_train_2016)
df_train_2017 = add_date_features(df_train_2017)

In [54]:
df_pred_2016 = add_date_features(df_pred_2016)
df_pred_2017 = add_date_features(df_pred_2017)

### needs to be generated from full data set

In [56]:
parcel_location_variables = ['regionidneighborhood',
                                 'regionidzip', 
                                 'regionidcity', 
                                 #'censustrack',
                                 'rawcensustractandblock']

# through for all 2016 data:
for region in parcel_location_variables:

    #### COUNT OF PROPERTIES ####
    # number of properties in the zipcode
    group = df_pred_2016[region].value_counts().to_dict()
    df_pred_2016['f_num_n_prop_in_'+region] = df_pred_2016[region].map(group)
    df_train_2016['f_num_n_prop_in_'+region] = df_train_2016[region].map(group)
    
    ##### HOW NEW IS THIS BUILDING COMPARING TO OTHER BUILDINGS #####
    group = df_pred_2016.groupby(region)['yearbuilt'].aggregate('median').to_dict()
    df_pred_2016['f_cat_median_year_in_'+region] = df_pred_2016[region].map(group)
    df_train_2016['f_cat_median_year_in_'+region] = df_train_2016[region].map(group)
    
    df_pred_2016['f_num_how_new_in_'+region] = df_pred_2016['yearbuilt'] - df_pred_2016['f_cat_median_year_in_'+region]
    df_train_2016['f_num_how_new_in_'+region] = df_train_2016['yearbuilt'] - df_train_2016['f_cat_median_year_in_'+region]

    # Neighborhood latitude and longitude
    group = df_pred_2016.groupby(region)['latitude'].aggregate('median').to_dict()
    df_pred_2016['f_num_median_lat_in_'+region] = df_pred_2016[region].map(group)
    df_train_2016['f_num_median_lat_in_'+region] = df_train_2016[region].map(group)
    
    
    group = df_pred_2016.groupby(region)['longitude'].aggregate('median').to_dict()
    df_pred_2016['f_num_median_lon_in_'+region] = df_pred_2016[region].map(group)
    df_train_2016['f_num_median_lon_in_'+region] = df_train_2016[region].map(group)

    #Average structuretaxvaluedollarcnt by city
    group = df_pred_2016.groupby(region)['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
    df_pred_2016['f_num_Avg-structuretaxvaluedollarcnt'] = df_pred_2016['regionidcity'].map(group)
    df_train_2016['f_num_Avg-structuretaxvaluedollarcnt'] = df_train_2016['regionidcity'].map(group)

    #Deviation away from average
    df_pred_2016['f_num_Dev-structuretaxvaluedollarcnt'] = abs((df_pred_2016['structuretaxvaluedollarcnt'] - df_pred_2016['f_num_Avg-structuretaxvaluedollarcnt']))/df_pred_2016['f_num_Avg-structuretaxvaluedollarcnt']
    df_train_2016['f_num_Dev-structuretaxvaluedollarcnt'] = abs((df_train_2016['structuretaxvaluedollarcnt'] - df_train_2016['f_num_Avg-structuretaxvaluedollarcnt']))/df_train_2016['f_num_Avg-structuretaxvaluedollarcnt']



In [58]:
# and do it for 2017
for region in parcel_location_variables:

    #### COUNT OF PROPERTIES ####
    # number of properties in the zipcode
    group = df_pred_2017[region].value_counts().to_dict()
    df_pred_2017['f_num_n_prop_in_'+region] = df_pred_2017[region].map(group)
    df_train_2017['f_num_n_prop_in_'+region] = df_train_2017[region].map(group)
    
    ##### HOW NEW IS THIS BUILDING COMPARING TO OTHER BUILDINGS #####
    group = df_pred_2017.groupby(region)['yearbuilt'].aggregate('median').to_dict()
    df_pred_2017['f_cat_median_year_in_'+region] = df_pred_2017[region].map(group)
    df_train_2017['f_cat_median_year_in_'+region] = df_train_2017[region].map(group)
    
    df_pred_2017['f_num_how_new_in_'+region] = df_pred_2017['yearbuilt'] - df_pred_2017['f_cat_median_year_in_'+region]
    df_train_2017['f_num_how_new_in_'+region] = df_train_2017['yearbuilt'] - df_train_2017['f_cat_median_year_in_'+region]

    # Neighborhood latitude and longitude
    group = df_pred_2017.groupby(region)['latitude'].aggregate('median').to_dict()
    df_pred_2017['f_num_median_lat_in_'+region] = df_pred_2017[region].map(group)
    df_train_2017['f_num_median_lat_in_'+region] = df_train_2017[region].map(group)
    
    
    group = df_pred_2017.groupby(region)['longitude'].aggregate('median').to_dict()
    df_pred_2017['f_num_median_lon_in_'+region] = df_pred_2017[region].map(group)
    df_train_2017['f_num_median_lon_in_'+region] = df_train_2017[region].map(group)

    #Average structuretaxvaluedollarcnt by city
    group = df_pred_2017.groupby(region)['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
    df_pred_2017['f_num_Avg-structuretaxvaluedollarcnt'] = df_pred_2017['regionidcity'].map(group)
    df_train_2017['f_num_Avg-structuretaxvaluedollarcnt'] = df_train_2017['regionidcity'].map(group)

    #Deviation away from average
    df_pred_2017['f_num_Dev-structuretaxvaluedollarcnt'] = abs((df_pred_2017['structuretaxvaluedollarcnt'] - df_pred_2017['f_num_Avg-structuretaxvaluedollarcnt']))/df_pred_2017['f_num_Avg-structuretaxvaluedollarcnt']
    df_train_2017['f_num_Dev-structuretaxvaluedollarcnt'] = abs((df_train_2017['structuretaxvaluedollarcnt'] - df_train_2017['f_num_Avg-structuretaxvaluedollarcnt']))/df_train_2017['f_num_Avg-structuretaxvaluedollarcnt']

### Now additional features only applicable to transaction data

### 2. time related features
Here the features are generated from SQL.

In [33]:
!psql -d zillow -a -f ../scripts/feature_engineering_time.sql

DROP TABLE IF EXISTS transactions_2016;
DROP TABLE
CREATE TABLE transactions_2016 (
	parcelid bigint, 
	logerror double precision,
	transactiondate varchar
);
CREATE TABLE
DROP TABLE IF EXISTS transactions_2017;
DROP TABLE
COPY transactions_2016
FROM '/Users/dai_li/Workspace/personal/Competitions/zillow/data/train_2016_v2.csv' DELIMITER ',' CSV HEADER;
COPY 90275
CREATE TABLE transactions_2017 (
	parcelid bigint, 
	logerror double precision,
	transactiondate varchar
);
CREATE TABLE
COPY transactions_2017
FROM '/Users/dai_li/Workspace/personal/Competitions/zillow/data/train_2017.csv' DELIMITER ',' CSV HEADER;
COPY 77613
DROP TABLE IF EXISTS transactions;
DROP TABLE
CREATE TABLE transactions AS
SELECT * FROM transactions_2016
UNION ALL
SELECT * FROM transactions_2017;
SELECT 167888
DROP TABLE IF EXISTS tmp_additional_temporal_information;
DROP TABLE
CREATE TABLE tmp_additional_temporal_information AS
SELECT 
    t.parcelid
    , t.logerror
    , t.transactiondate
    , substring(transact

In [34]:
d_features = pd.read_csv('../data/2nd_monthly_transactions_features.csv')

In [86]:
df_train = pd.concat([df_train_2016, df_train_2017], axis = 0)

In [87]:
df_train['f_num_month'] = pd.to_datetime(df_train.transactiondate).dt.month.astype(int)
df_train['f_num_quarter'] = pd.to_datetime(df_train.transactiondate).dt.quarter.astype(int)

In [121]:
str(4)[:4]

'4'

In [131]:
for m in [201610, 201611, 201612, 201710, 201711, 201712]:
    year = str(m)[:4]
    month = str(m)[4:]
    
    # avg log error, monthly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('mean').to_dict()
    df_train['f_num_monthly_avg_logerror'] = df_train['f_num_month'].map(group)
    eval('df_pred_'+str(m))['f_num_monthly_avg_logerror'] = group[int(month)]
                                   
    # std dev log error, monthly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('std').to_dict()
    df_train['f_num_monthly_stddev_logerror'] = df_train['f_num_month'].map(group)
    eval('df_pred_'+str(m))['f_num_monthly_stddev_logerror'] = group[int(month)]

    # avg log error, quarterly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('mean').to_dict()
    df_train['f_num_quarterly_avg_logerror'] = df_train['f_num_quarter'].map(group)
    eval('df_pred_'+str(m))['f_num_quarterly_avg_logerror'] = group[4]
    
    # std dev log error, monthly
    group = df_train.groupby(df_train.f_num_month)['logerror'].aggregate('std').to_dict()
    df_train['f_num_quarterly_stddev_logerror'] = df_train['f_num_quarter'].map(group)
    eval('df_pred_'+str(m))['f_num_quarterly_stddev_logerror'] = group[4]
    

# save the cleaned data sets

In [132]:
df_train.to_csv('../tmp/train_full.csv', index=False, header=True)

In [135]:
df_pred_2016.to_csv('../tmp/pred_2016.csv', index=False, header=True)

In [136]:
df_pred_2017.to_csv('../tmp/pred_2017.csv', index=False, header=True)

# Generate masks

In [137]:
df = df_train

In [138]:
all_fields = set(df.columns)

# these are fields that are used to identify fields
identifiers = set(['transactiondate', 'parcelid'])

# log error that we want to model
label = set(['logerror'])

# the following are categorical features
feats_objects = set(
 ['taxdelinquencyflag',
 'propertycountylandusecode',
 'propertyzoningdesc',
 'fireplaceflag',
 'hashottuborspa']
)

# the following are numerical features that should be treated as categorical features
feats_categorical_as_numeric = set([
    'airconditioningtypeid',
    'architecturalstyletypeid',
    'buildingqualitytypeid',
    'buildingclasstypeid',
    'decktypeid',
    'fips',
    'heatingorsystemtypeid',
    'propertylandusetypeid',
    'regionidcounty',
    'regionidcity',
    'regionidzip',
    'regionidneighborhood',
    'storytypeid',
    'typeconstructiontypeid',
])


# the rest are numeric features
feats_numeric = set([
    'basementsqft',
    'bathroomcnt',
    'bedroomcnt',
    'calculatedbathnbr',
    'threequarterbathnbr',
    'finishedfloor1squarefeet',
    'calculatedfinishedsquarefeet',
    'finishedsquarefeet6',
    'finishedsquarefeet12',
    'finishedsquarefeet13',
    'finishedsquarefeet15',
    'finishedsquarefeet50',
    'fireplacecnt',
    'fullbathcnt',
    'garagecarcnt',
    'garagetotalsqft',
    'hashottuborspa',
    'lotsizesquarefeet',
    'numberofstories',
    'poolcnt',
    'poolsizesum',
    'pooltypeid10',
    'pooltypeid2',
    'pooltypeid7',
    'roomcnt',
    'unitcnt',
    'yardbuildingsqft17',
    'yardbuildingsqft26',
    'taxvaluedollarcnt',
    'structuretaxvaluedollarcnt',
    'landtaxvaluedollarcnt',
    'taxamount',
    'latitude',
    'longitude',
    'yearbuilt',
    'assessmentyear',
    'taxdelinquencyyear',
    'rawcensustractandblock',
    'censustractandblock',
])

feats_numerics_feature_engineered = set([col for col in df.columns if 'f_num' in col or '_logerror' in col])


feats_categorical_feature_engineered = set([col for col in df.columns if 'f_cat' in col])


# fields that are thrown away for now
feats_for_consideration_later = set([
])

In [139]:
feats_categorical = feats_objects | feats_categorical_feature_engineered
feats_numeric = feats_numeric | feats_numerics_feature_engineered 
feats = feats_categorical | feats_numeric | feats_categorical_as_numeric

feats_categorical_as_numeric = list(feats_categorical_as_numeric)
feats_categorical = list(feats_categorical)
feats_numeric = list(feats_numeric)
feats = list(feats)

# save results to pick files

In [140]:
pickle.dump(feats_categorical_as_numeric, open('../tmp/feats_categorical_as_numeric.pkl', 'w'))
pickle.dump(feats_categorical, open('../tmp/feats_categorical.pkl', 'w'))
pickle.dump(feats_numeric, open('../tmp/feats_numeric.pkl', 'w'))
pickle.dump(feats, open('../tmp/feats.pkl', 'w'))

In [141]:
mask_train = (df.transactiondate < '2019-01-01')
mask_validation = (df.transactiondate >= '2016-10-01') & (df.transactiondate < '2017-01-01')
# mask_prediction = ~df.parcelid.isnull()

In [142]:
pickle.dump(mask_train, open('../tmp/mask_train.pkl', 'w'))
pickle.dump(mask_validation, open('../tmp/mask_validation.pkl', 'w'))
# pickle.dump(mask_prediction, open('../tmp/mask_prediction.pkl', 'w'))

In [144]:
feats

['finishedfloor1squarefeet',
 'numberofstories',
 'f_num_longitude-round',
 'f_cat_median_year_in_regionidneighborhood',
 'f_num_locatiof_num_2round',
 'regionidzip',
 'f_num_avg_logerror',
 'buildingclasstypeid',
 'regionidcounty',
 'taxdelinquencyflag',
 'f_num_ValueProp',
 'pooltypeid10',
 'f_num_LivingAreaProp2',
 'f_num_how_new_in_regionidcity',
 'f_num_n_prop_in_regionidcity',
 'structuretaxvaluedollarcnt',
 'poolsizesum',
 'f_num_n_prop_in_rawcensustractandblock',
 'f_num_median_lon_in_regionidneighborhood',
 'finishedsquarefeet6',
 'f_num_monthly_stddev_logerror',
 'calculatedfinishedsquarefeet',
 'latitude',
 'bedroomcnt',
 'landtaxvaluedollarcnt',
 'hashottuborspa',
 'f_num_locatiof_num_2',
 'buildingqualitytypeid',
 'f_num_median_lat_in_regionidcity',
 'f_num_GarPoolAC',
 'propertylandusetypeid',
 'regionidneighborhood',
 'f_num_stddev_logerror',
 'decktypeid',
 'f_cat_median_year_in_rawcensustractandblock',
 'heatingorsystemtypeid',
 'architecturalstyletypeid',
 'rawcensust